In [1]:
# Importando bibliotecas
from functions import *
import pandas as pd
import locale
from pathlib import Path
from datetime import datetime
import duckdb
import gc
import numpy as np
import warnings
import logging
import shutil

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA

logging.basicConfig(level=logging.WARNING, format='%(message)s')

warnings.filterwarnings("ignore")

timer = Temporizador()
timer.iniciar()

locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.expand_frame_repr', False)

# Detecta se o script está sendo executado de um .py ou de um notebook
try:
    caminho_base = Path(__file__).resolve().parent
except NameError:
    # __file__ não existe em Jupyter ou ambiente interativo
    caminho_base = Path.cwd()

pasta_input_parquet = caminho_base.parent / '01_INPUT_PIPELINE/01_BD_PARQUET'
arquivo_input_regras_negocio = caminho_base.parent / '01_INPUT_PIPELINE/02_REGRAS_NEGOCIO/KRONA_REGRAS.xlsm'
pasta_staging_parquet = caminho_base.parent / '02_STAGING_PARQUET' # Armazena arquivos parquet com tratamentos, aplicações de regras, depara, etc
pasta_input_painel = caminho_base.parent / '03_INPUT_PAINEL' # Armazena arquivos que serão consumidos no painel de S&OP para os gerentes
pasta_painel = caminho_base.parent / '05_PAINEL'

# # Eliminar arquivos das pastas de 02_STAGING_PARQUET e 03_INPUT_PAINEL que serão regenerados
# pastas_para_limpar = [
#     pasta_staging_parquet,
#     pasta_input_painel,
# ]

# for pasta in pastas_para_limpar:
#     if pasta.exists() and pasta.is_dir():
#         for item in pasta.iterdir():
#             if item.is_file() or item.is_symlink():
#                 item.unlink()
#             elif item.is_dir():
#                 shutil.rmtree(item)

print("✅ Mapeamento de pastas concluído com sucesso!")

✅ Mapeamento de pastas concluído com sucesso!


In [2]:

# Carregar dados arquivo KRONA_REGRAS
caminho_arquivo = arquivo_input_regras_negocio

#-----------------------------------------------------------------------#
#--------------- Carregar produtos eliminar ----------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'PRODUTOS_ELIMINAR'
df_produtos_eliminar = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'COD_PROD': str})
df_produtos_eliminar['COD_PROD'] = df_produtos_eliminar['COD_PROD'].astype(str)
df_produtos_eliminar = df_produtos_eliminar.drop_duplicates(subset=['COD_PROD'])
df_produtos_eliminar = df_produtos_eliminar[df_produtos_eliminar['COD_PROD'].notna()].reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carregar Regionais Gestor ------------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'REGIONAIS_GESTOR'
df_regionais_gestor = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_regionais_gestor = df_regionais_gestor.drop_duplicates(subset=['REGIONAL', 'REGIONAL_GESTOR'])
df_regionais_gestor = df_regionais_gestor[df_regionais_gestor['REGIONAL'].notna()].reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carrregar Demanda Lançamento Novos Produtos ------------#
#-----------------------------------------------------------------------#
guia_excel = 'PRODUTOS_LANCAMENTOS'
df_produtos_lancamento = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
# 🚨 VALIDAR SE EXISTEM DADOS
if df_produtos_lancamento.empty:
    raise ValueError(
        "❌ ERRO: Nenhuma informação foi encontrada na aba PRODUTOS_LANCAMENTOS.\n"
        "➡️ Verifique se a planilha possui dados válidos antes de executar o pipeline."
    )
df_produtos_lancamento['JANELA LANÇAMENTO'] = df_produtos_lancamento['JANELA LANÇAMENTO'].astype(str).str.strip()
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['JANELA LANÇAMENTO'] != ''].reset_index(drop=True)
df_produtos_lancamento.rename(columns={'COD': 'COD_PROD'}, inplace=True)
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['COD_PROD'].notna()].reset_index(drop=True)
df_produtos_lancamento['COD_PROD'] = df_produtos_lancamento['COD_PROD'].astype(str)

# Identifica colunas com datas válidas
col_datas = []
for col in df_produtos_lancamento.columns:
    try:
        pd.to_datetime(col, dayfirst=True, errors='raise')
        col_datas.append(col)
    except (ValueError, TypeError):
        continue

colunas_validas = ['COD_PROD'] + \
                    [col for col in df_produtos_lancamento.columns if 'CD:' in str(col)] + \
                    col_datas
df_produtos_lancamento = df_produtos_lancamento[[col for col in colunas_validas if col in df_produtos_lancamento.columns]]

# Transforma datas em linhas
df_produtos_lancamento = df_produtos_lancamento.melt(
    id_vars=[col for col in df_produtos_lancamento.columns if col not in col_datas],
    value_vars=col_datas,
    var_name='PERIODO',
    value_name='VALOR'
)
df_produtos_lancamento = df_produtos_lancamento[df_produtos_lancamento['VALOR'].notna()].reset_index(drop=True)

# Multiplica colunas CD pelo VALOR
colunas_cd = [col for col in df_produtos_lancamento.columns if 'CD:' in str(col)]
for col in colunas_cd:
    df_produtos_lancamento[col] = df_produtos_lancamento[col] * df_produtos_lancamento['VALOR']
df_produtos_lancamento.drop(columns=['VALOR'], inplace=True)

# Transforma colunas CD em linhas
df_produtos_lancamento = df_produtos_lancamento.melt(
    id_vars=[col for col in df_produtos_lancamento.columns if col not in colunas_cd],
    value_vars=colunas_cd,
    var_name='CD',
    value_name='QTD'
)

# Carregar a tabela DE_PARA_CD para fazer de para das unidades CD
guia_excel = 'DE_PARA_CD'
df_de_para_cd = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'DE': str, 'PARA': str})

# Fazer merge do df_produtos_lancamento com o df_de_para_cd para substituir os códigos CD
df_produtos_lancamento = df_produtos_lancamento.merge(
    df_de_para_cd,
    left_on='CD',
    right_on='DE',
    how='left'
)

# Atualizar a coluna CD com os valores da coluna PARA
df_produtos_lancamento['CD'] = df_produtos_lancamento['PARA']

# Remover colunas DE e PARA
df_produtos_lancamento.drop(columns=['DE', 'PARA'], inplace=True)

# Coluna CD transformar em maiusculo e remover espaços em branco
df_produtos_lancamento['CD'] = df_produtos_lancamento['CD'].astype(str).str.upper().str.strip()

# Salvar em parquet para uso posterior
df_produtos_lancamento.to_parquet(pasta_staging_parquet / 'DEMANDA_LANCAMENTO_PRODUTOS_KRONA.parquet', index=False)

#-----------------------------------------------------------------------#
#---------------Carregar Regionais Construtora -------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'REGIONAIS_CONSTRUTORA'
df_regionais_construtora = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_regionais_construtora = df_regionais_construtora.drop_duplicates(subset=['REGIONAL BASE', 'REGIONAL ATUALIZADA'])

#-----------------------------------------------------------------------#
#---------------Carregar Clientes para planejamento de Demanda----------#
#-----------------------------------------------------------------------#
guia_excel = 'CLIENTES_DEMANDA'
df_clientes_plan_demanda = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'Cod_Grupo_Cliente': str})
df_clientes_plan_demanda = df_clientes_plan_demanda.drop_duplicates(subset=['Cod_Grupo_Cliente'])
df_clientes_plan_demanda = df_clientes_plan_demanda[df_clientes_plan_demanda['Cod_Grupo_Cliente'].notna()].reset_index(drop=True)

# Converter a coluna de clientes para set para acelerar o isin
lista_clientes_plan_demanda = set(df_clientes_plan_demanda['Cod_Grupo_Cliente'])

# Unir COD_PROD de df_produtos_lancamento e df_produtos_eliminar, formar uma unica lista de produtos a eliminar, e remover do df_fato_vendas_krona
# produtos_a_eliminar = pd.concat([df_produtos_eliminar[['COD_PROD']], df_produtos_lancamento[['COD_PROD']]]).drop_duplicates().reset_index(drop=True)
# FIXME: Retirei os produtos de lançamento da lista de exclusão conforme solicitação da Anna no WORD
produtos_a_eliminar = df_produtos_eliminar[['COD_PROD']].drop_duplicates().reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carregar DIRECIONA_CLIENTES_REGIONAL--------------------#
#-----------------------------------------------------------------------#
guia_excel = 'DIRECIONA_CLIENTES_REGIONAL'
df_direc_cli_regional = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine', dtype={'COD_GRUPO_CLIENTE': str, 'COD_CLIENTE': str})
df_direc_cli_regional = df_direc_cli_regional[df_direc_cli_regional['COD_CLIENTE'].notna()].reset_index(drop=True)

#-----------------------------------------------------------------------#
#---------------Carregar PERIODO_PREVISAO-------------------------------#
#-----------------------------------------------------------------------#
guia_excel = 'PERIODO_PREVISAO'
df_periodo_previsao = pd.read_excel(caminho_arquivo, sheet_name=guia_excel, engine='calamine')
df_periodo_previsao = df_periodo_previsao[df_periodo_previsao['PERIODO_PROJECAO'].notna()].reset_index(drop=True)
df_periodo_previsao = df_periodo_previsao.drop_duplicates(subset=['PERIODO_PROJECAO'])

print("✅ Importação e tratamento de dados do arquivo KRONA_REGRAS, concluídos com sucesso!")

✅ Importação e tratamento de dados do arquivo KRONA_REGRAS, concluídos com sucesso!


In [3]:
# Script para eliminar duplicação de Chv_Cliente no Dim_Clientes_Krona, conforme orientado por Marcos TI, criamos essa rotina para encontrar as duplicações, eliminar e gerar novo Parquet sem duplicações.

# Carregar o Parquet
df_dim_cli_krona = pd.read_parquet(pasta_input_parquet / "Dim_Clientes_Krona.parquet")

# Eliminar duplciações mantendo a primeira ocorrência
df_dim_cli_krona = df_dim_cli_krona.drop_duplicates(subset=["Chv_Cliente"], keep='first').reset_index(drop=True)

# Gerar novo Parquet sem duplicações
df_dim_cli_krona.to_parquet(pasta_input_parquet / "Dim_Clientes_Krona.parquet", index=False)

del df_dim_cli_krona
gc.collect()

0

In [4]:
# Where Des_Origem = 'Krona'
#   And Cod_Empresa IN ('01','05','08','0802','10')
#   And Dat_Emissao_Venda >= '01/01/2018';

# Na dimensão de Produto tem o seguinte filtro:
# Where Cod_Empresa IN ('01','05','08','0802','10')

# Nos casos de pedidos cancelados, consideramos apenas o que foi realmente vendido ao cliente:
# Se o pedido foi cancelado por completo, não aparece nenhuma venda.
# Se apenas uma parte foi cancelada, consideramos somente a parte que foi vendida.
# Além disso, também existem os bloqueios de pedidos, que representam a “etapa” em que o pedido se encontra. Nesses casos, é importante definir quais bloqueios devem ser considerados nessa análise.

# falei com o André aqui pelo chat, e verificou com a Aline
# a principio utiliza os códigos de bloqueio 80, 90 , 95 e 99

# No arquivo parquet, precisamos filtrar o campo Des_Origem = "Krona". O motivo é que existem pedidos faturados pelo Protheus que contam no resultado da Viqua, e este aplicativo "CML - Vendas" do Qlik Sense traz apenas os pedidos que geram resultado para a Krona

# Consolidando as informações de fontes em Parquet
# Conslidando as informações de fontes em Parquet
empresa = 'Krona'
fact = (pasta_input_parquet / "Fato_Vendas_Krona.parquet").as_posix()
prod = (pasta_input_parquet / "Dim_Produtos_Vendas_Krona.parquet").as_posix()
cli  = (pasta_input_parquet / "Dim_Clientes_Krona.parquet").as_posix()
vend = (pasta_input_parquet / "Dim_Vendedores_Krona.parquet").as_posix()

sql = f"""
WITH
fato AS (
  SELECT
    Cod_Produto,
    Chv_Cliente,
    Chv_Vendedor,
    DATE_TRUNC(
      'month',
      CAST(
        COALESCE(
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%Y-%m-%d'),
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%d/%m/%Y')
        ) AS DATE
      )
    ) AS PERIODO,
    TRIM(Nom_Empresa) AS EMPRESA,
    SUM(TRY_CAST(Qtd_Venda AS DOUBLE)) AS QTD_VENDA,
    SUM(TRY_CAST(Qtd_Peso_Venda AS DOUBLE)) AS VOL_VENDA
  FROM parquet_scan('{fact}')
  WHERE UPPER(TRIM(Nom_Empresa)) LIKE '%{empresa.strip().upper()}%'
    AND UPPER(TRIM(Des_Origem))  LIKE '%{empresa.strip().upper()}%'
    AND Cod_Empresa IN ('01','05','08','0802','10')
    AND TRY_CAST(NULLIF(TRIM(Cod_Bloqueio), '') AS INTEGER) IN (80,90,95,99)
    AND TRY_CAST(Qtd_Venda AS DOUBLE) > 0
    AND Dat_Entrega_Venda IS NOT NULL
    AND TRIM(Dat_Entrega_Venda) <> ''
    AND COALESCE(
      TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%Y-%m-%d'),
      TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%d/%m/%Y')
    ) >= DATE '2022-01-01'
    -- EXCLUI MÊS ATUAL
    AND DATE_TRUNC(
      'month',
      CAST(
        COALESCE(
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%Y-%m-%d'),
          TRY_STRPTIME(TRIM(Dat_Entrega_Venda), '%d/%m/%Y')
        ) AS DATE
      )
    ) < DATE_TRUNC('month', CURRENT_DATE)
  GROUP BY Cod_Produto, Chv_Cliente, Chv_Vendedor, PERIODO, EMPRESA
),
prod AS (
  SELECT
    Cod_Produto,
    TRIM(Des_Produto) AS Des_Produto,
    Cod_Familia,
    TRIM(Des_Familia) AS Des_Familia,
    Cod_Linha,
    TRIM(Des_Linha) AS Des_Linha,
    TRIM(Nom_Empresa) AS EMPRESA,
    TRY_CAST(Num_Peso AS DOUBLE) AS PESO_UNIT
  FROM parquet_scan('{prod}')
  WHERE Des_Linha IS NOT NULL
    AND TRIM(Des_Linha) <> ''
    AND Cod_Empresa IN ('01','05','08','0802','10')
),
cli AS (
  SELECT
    Chv_Cliente,
    TRIM(Nom_Cliente) AS NOME_CLIENTE,
    TRIM(Nom_Empresa) AS EMPRESA,
    Chv_Vendedor_Cliente,
    TRIM(Des_Segmento) AS SEGMENTO,
    -- Corrige COD_GRUPO_CLIENTE: se vazio, usa COD_CLIENTE
    CASE
      WHEN TRIM(Cod_Grupo_Cliente) = '' OR Cod_Grupo_Cliente IS NULL
      THEN TRIM(SPLIT_PART(Chv_Cliente, '|', 2))
      ELSE TRIM(Cod_Grupo_Cliente)
    END AS COD_GRUPO_CLIENTE,
    -- Corrige DESC_GRUPO_E_CLIENTE: se vazio, usa NOME_CLIENTE
    CASE
      WHEN TRIM(Des_Grupo_e_Cliente) = '' OR Des_Grupo_e_Cliente IS NULL
      THEN TRIM(Nom_Cliente)
      ELSE TRIM(Des_Grupo_e_Cliente)
    END AS DESC_GRUPO_E_CLIENTE
  FROM parquet_scan('{cli}')
),
vend AS (
  SELECT
    Chv_Vendedor,
    TRIM(Des_Regiao) AS Des_Regiao
  FROM parquet_scan('{vend}')
),
final AS (
  SELECT
    f.EMPRESA,
    TRIM(SPLIT_PART(c.Chv_Cliente, '|', 2)) AS COD_CLIENTE,
    c.NOME_CLIENTE,
    c.COD_GRUPO_CLIENTE,
    c.DESC_GRUPO_E_CLIENTE,
    c.SEGMENTO,
    f.Cod_Produto AS COD_PROD,
    p.Des_Produto AS DESC_PRODUTO,
    CAST(p.Cod_Familia AS VARCHAR) || ' - ' || p.Des_Familia AS FAMILIA,
    CAST(p.Cod_Linha   AS VARCHAR) || ' - ' || p.Des_Linha   AS LINHA,
    v1.Des_Regiao AS REGIAO_CLIENTE,
    v2.Des_Regiao AS REGIAO_MOVIMENTO,
    f.PERIODO,
    f.QTD_VENDA,
    f.VOL_VENDA,
    f.VOL_VENDA / f.QTD_VENDA AS PESO_UNIT
  FROM fato f
  LEFT JOIN prod p ON f.Cod_Produto = p.Cod_Produto AND f.EMPRESA = p.EMPRESA
  LEFT JOIN cli  c ON f.Chv_Cliente = c.Chv_Cliente AND f.EMPRESA = c.EMPRESA
  LEFT JOIN vend v1 ON c.Chv_Vendedor_Cliente = v1.Chv_Vendedor
  LEFT JOIN vend v2 ON f.Chv_Vendedor         = v2.Chv_Vendedor
)
SELECT
  UPPER(EMPRESA) AS EMPRESA,
  COD_CLIENTE,
  NOME_CLIENTE,
  COD_GRUPO_CLIENTE,
  DESC_GRUPO_E_CLIENTE,
  SEGMENTO,
  COD_PROD,
  DESC_PRODUTO,
  FAMILIA,
  LINHA,
  PESO_UNIT,
  REGIAO_CLIENTE,
  REGIAO_MOVIMENTO,
  PERIODO,
  QTD_VENDA,
  VOL_VENDA
FROM final
"""
df_vendas_krona_silver = duckdb.query(sql).to_df()

# Salvar em parquet
df_vendas_krona_silver.to_parquet(pasta_staging_parquet / "df_vendas_krona_silver.parquet", index=False)

# del df_vendas_krona_silver
# gc.collect()

print("✅ Carregamento de df_vendas_krona_silver concluído com sucesso!")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Carregamento de df_vendas_krona_silver concluído com sucesso!


In [5]:
# ============================================================
# 1. Criando coluna REGIONAL copiando a coluna REGIAO_CLIENTE 
#    no df_vendas_krona. 
#    Onde o segmento contém CONSTRUTORA ou INSTALADOR, buscar 
#    na tabela de regionais_construtora a regional atualizada.
# ============================================================

# Carregar o df_vendas_krona_silver do parquet
df_vendas_krona_gold = pd.read_parquet(pasta_staging_parquet / "df_vendas_krona_silver.parquet")

# Cria a tabela de de-para das regionais (já registrada no engine)
duckdb.register("vendas", df_vendas_krona_gold)
duckdb.register("map_reg", df_regionais_construtora[['REGIONAL BASE','REGIONAL ATUALIZADA']])
duckdb.register("direc_cli_regional", df_direc_cli_regional[['COD_CLIENTE', 'REGIONAL']])

sql = """
WITH base AS (
  SELECT
    v.*,
    -- Substitui valores vazios de REGIAO_CLIENTE por REGIAO_MOVIMENTO
    COALESCE(NULLIF(v.REGIAO_CLIENTE,''), v.REGIAO_MOVIMENTO) AS RC_FIX,
    UPPER(v.SEGMENTO) AS SEG_UP,
    UPPER(v.REGIAO_CLIENTE) AS RC,
    UPPER(v.REGIAO_MOVIMENTO) AS RM
  FROM vendas v
),

ajuste AS (
  SELECT
    b.*,
    CASE
      -- >>> ADICIONADO: override por cliente (se existir na df_direc_cli_regional)
      WHEN d.REGIONAL IS NOT NULL AND d.REGIONAL <> '' THEN d.REGIONAL
      
      -- 1) Se SEGMENTO contém CONSTRUTORA ou INSTALADOR => usa de-para
      WHEN b.SEG_UP LIKE '%CONSTRUTORA%' OR b.SEG_UP LIKE '%INSTALADOR%'
        THEN COALESCE(m."REGIONAL ATUALIZADA", b.RC_FIX)
      -- ============================================================
      -- 2. Converter TELEVENDAS - Regras para definir REGIONAL:
      --    REGIONAL = CONSTRUTORA => REGIONAL_CONSTRUTORA
      --    REGIAO_CLIENTE = TELEVENDAS e REGIAO_MOVIMENTO = TELEVENDAS => TELEVENDAS
      --    REGIAO_CLIENTE != TELEVENDAS e REGIAO_MOVIMENTO = TELEVENDAS => TELEVENDAS
      --    REGIAO_CLIENTE = TELEVENDAS e REGIAO_MOVIMENTO != TELEVENDAS => REGIAO_MOVIMENTO
      --    Caso contrário => REGIAO_CLIENTE
      -- ============================================================
      WHEN b.RC='TELEVENDAS' AND b.RM='TELEVENDAS' THEN 'TELEVENDAS'
      WHEN b.RC<>'TELEVENDAS' AND b.RM='TELEVENDAS' THEN 'TELEVENDAS'
      WHEN b.RC='TELEVENDAS' AND b.RM<>'TELEVENDAS' THEN b.RM
      ELSE b.RC_FIX
    END AS REGIONAL
  FROM base b
  LEFT JOIN map_reg m
    ON m."REGIONAL BASE" = b.REGIAO_CLIENTE
  -- >>> ADICIONADO: join com regional direcionada por cliente
  LEFT JOIN direc_cli_regional d
    ON d.COD_CLIENTE = b.COD_CLIENTE
)

-- ============================================================
-- Resultado final consolidado
-- ============================================================
SELECT
  EMPRESA,
  COD_CLIENTE,
  NOME_CLIENTE,
  COD_GRUPO_CLIENTE,
  DESC_GRUPO_E_CLIENTE,
  COD_PROD,
  DESC_PRODUTO,
  FAMILIA,
  LINHA,
  REGIONAL,
  PERIODO,
  SUM(QTD_VENDA) AS QTD_VENDA,
  SUM(VOL_VENDA) AS VOL_VENDA
FROM ajuste
-- WHERE REGIONAL IS NOT NULL AND REGIONAL <> ''
GROUP BY
  EMPRESA,
  COD_CLIENTE,
  NOME_CLIENTE,
  COD_GRUPO_CLIENTE,
  DESC_GRUPO_E_CLIENTE,
  COD_PROD,
  DESC_PRODUTO,
  FAMILIA,
  LINHA,
  REGIONAL,
  PERIODO
"""

# Executa no DuckDB
df_vendas_krona_gold= duckdb.query(sql).to_df()

# Inserir REGIONAL_GESTOR no df_vendas_krona
df_vendas_krona_gold = pd.merge(
    df_vendas_krona_gold,
    df_regionais_gestor,
    left_on='REGIONAL',
    right_on='REGIONAL',
    how='left'
)

colunas_ordenadas = [
    "EMPRESA",
    "COD_CLIENTE",
    "NOME_CLIENTE",
    "COD_GRUPO_CLIENTE",
    "DESC_GRUPO_E_CLIENTE",
    "COD_PROD",
    "DESC_PRODUTO",
    "FAMILIA",
    "LINHA",
    "REGIONAL",
    "REGIONAL_GESTOR",
    "PERIODO",
    "QTD_VENDA",
    "VOL_VENDA"
]

df_vendas_krona_gold = df_vendas_krona_gold[colunas_ordenadas]

# Salvar df_vendas_krona_gold em Parquet para salvar as alterações, filtros e regras aplicadas no histórico, otimizando memória e garantindo rastreabilidade
df_vendas_krona_gold.to_parquet(pasta_staging_parquet / "df_vendas_krona_gold.parquet", index=False)

del df_vendas_krona_gold
gc.collect()

print("✅ Organização de Regionais e Inserção de Regional Gestor na df_vendas_krona_gold concluídos com sucesso!")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Organização de Regionais e Inserção de Regional Gestor na df_vendas_krona_gold concluídos com sucesso!


In [6]:
# Aplicar produtos a eliminar no df_vendas_krona_gold, e excluir os produtos listados na variavel produtos_a_eliminar vinda do arquivo de regras de negócio
df_vendas_krona = pd.read_parquet(pasta_staging_parquet / "df_vendas_krona_gold.parquet")
lista_produtos_eliminar = set(produtos_a_eliminar['COD_PROD'])
df_vendas_krona = df_vendas_krona[~df_vendas_krona['COD_PROD'].isin(lista_produtos_eliminar)]
df_vendas_krona.to_parquet(pasta_staging_parquet / "df_vendas_krona.parquet", index=False)

print("✅ Eliminação de produtos concluída!")

✅ Eliminação de produtos concluída!


In [7]:
# GERAR DIM_PESO_UNIT_VENDAS COM BASE NO df_vendas_krona

# 1. Criar coluna PESO_UNIT divindo VOL_VENDA por QTD_VENDA
df_dim_peso_unit_vendas = df_vendas_krona.copy()
df_dim_peso_unit_vendas["PESO_UNIT"] = df_dim_peso_unit_vendas["VOL_VENDA"] / df_dim_peso_unit_vendas["QTD_VENDA"]

# 2. Classificar PERIODO em ordem descrescente, COD_PROD e EMPRESA crescente
df_dim_peso_unit_vendas = df_dim_peso_unit_vendas.sort_values(by=["PERIODO", "COD_PROD", "EMPRESA"], ascending=[False, True, True])

# 3. Manter apenas as colunas COD_PROD, EMPRESA e PESO_UNIT
df_dim_peso_unit_vendas = df_dim_peso_unit_vendas[["COD_PROD", "EMPRESA", "PESO_UNIT"]]

# 4. Remover duplicatas
df_dim_peso_unit_vendas = df_dim_peso_unit_vendas.drop_duplicates(subset=["COD_PROD", "EMPRESA"], keep="first").reset_index(drop=True)

# 5. Salvar df_dim_peso_unit_vendas em Parquet
df_dim_peso_unit_vendas.to_parquet(pasta_staging_parquet / "df_dim_peso_unit_vendas.parquet", index=False)

In [8]:
# 🦆 Exportação de Dados Vendas para Planejamento Colaborativo
# 🎯 Objetivo: Exportar CSV para o Plano Colaborativo
df_vendas_krona['NIVEL_PLAN_DEMANDA'] = np.where(
    df_vendas_krona['COD_GRUPO_CLIENTE'].isin(lista_clientes_plan_demanda),
    'CLIENTE',
    'PRODUTO'
)

# Separa os DataFrames
df_hist_vend_PRODUTO = df_vendas_krona[df_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'PRODUTO']
df_hist_vend_CLIENTE = df_vendas_krona[df_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'CLIENTE']

# Eliminar coluna NIVEL_PLAN_DEMANDA
df_hist_vend_PRODUTO = df_hist_vend_PRODUTO.drop(columns=['NIVEL_PLAN_DEMANDA'])
df_hist_vend_CLIENTE = df_hist_vend_CLIENTE.drop(columns=['NIVEL_PLAN_DEMANDA'])

# Agrupar df_hist_vend_PRODUTO por REGIONAL_GESTOR, FAMILIA, PERIODO, VOL_VENDA
df_hist_vend_PRODUTO = df_hist_vend_PRODUTO.groupby(
    ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO'],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)

# Salva como CSV
df_hist_vend_PRODUTO.to_csv(
    pasta_input_painel / 'HIST_VENDA_KRONA_AGREGADO.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

# Agrupar df_hist_vend_CLIENTE por COD_GRUPO_CLIENTE, DESC_GRUPO_E_CLIENTE, REGIONAL_GESTOR, FAMILIA, PERIODO, VOL_VENDA
df_hist_vend_CLIENTE = df_hist_vend_CLIENTE.groupby(
    ["COD_GRUPO_CLIENTE","DESC_GRUPO_E_CLIENTE", "REGIONAL_GESTOR", 'REGIONAL', "FAMILIA", "PERIODO"],
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)

# Salva como CSV
df_hist_vend_CLIENTE.to_csv(
    pasta_input_painel / 'HIST_VENDA_KRONA_CLIENTE.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

In [9]:
# Gerar os arquivos com média de vendas para Planejamento Colaborativo Agregado
# Encontrar o primeiro dia do mês atual

colunas_agregadas = ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA']

hoje = datetime.today()
primeiro_dia_mes_atual = datetime(hoje.year, hoje.month, 1)

# Calcular o primeiro dia do mês de 6 meses atrás (excluindo mês atual)
primeiro_dia_6_meses_atras = (primeiro_dia_mes_atual - pd.DateOffset(months=6)).to_pydatetime()

# Filtrar apenas os últimos 6 meses (excluindo mês atual)
mask = (df_hist_vend_PRODUTO['PERIODO'] >= primeiro_dia_6_meses_atras) & (df_hist_vend_PRODUTO['PERIODO'] < primeiro_dia_mes_atual)
df_hist_vend_PRODUTO_ultimos_6_meses = df_hist_vend_PRODUTO.loc[mask].copy()

# Ordenar por data crescente
df_hist_vend_PRODUTO_ultimos_6_meses = df_hist_vend_PRODUTO_ultimos_6_meses.sort_values('PERIODO').reset_index(drop=True)

# DataFrame dos 3 meses mais recentes (últimos 3 meses do intervalo filtrado)
df_3_meses_mais_recentes = df_hist_vend_PRODUTO_ultimos_6_meses.copy()

# Identificar as 3 datas mais recentes (sem duplicar por linha)
meses_recentes = sorted(df_3_meses_mais_recentes['PERIODO'].unique())[-3:]

# Filtrar todas as linhas que pertencem a esses 3 meses
df_3_meses_mais_recentes = df_3_meses_mais_recentes[df_3_meses_mais_recentes['PERIODO'].isin(meses_recentes)].copy()

# Agrupa pelas colunas desejadas e calcula a média das colunas numéricas
df_3_meses_mais_recentes_media = df_3_meses_mais_recentes.groupby(colunas_agregadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 3 MESES' na coluna
df_3_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 3 MESES'

# Agrupamento fazendo média dos 6 meses
df_6_meses_mais_recentes_media = df_hist_vend_PRODUTO_ultimos_6_meses.copy()
df_6_meses_mais_recentes_media = df_6_meses_mais_recentes_media.groupby(colunas_agregadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 6 MESES' na coluna
df_6_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 6 MESES'

# Concatenar os DataFrames
df_media_vendas_PRODUTO = pd.concat([df_3_meses_mais_recentes_media, df_6_meses_mais_recentes_media], ignore_index=True)

# Pivotar a coluna MEDIA
df_media_vendas_PRODUTO = df_media_vendas_PRODUTO.pivot_table(
    index=colunas_agregadas,
    columns='MEDIA',
    values='VOL_VENDA',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Gerar o arquivo CSV
df_media_vendas_PRODUTO.to_csv(
    pasta_input_painel / 'MEDIA_VENDA_KRONA_AGREGADO.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

In [10]:

# Gerar os arquivos com média de vendas para Planejamento Colaborativo por Cliente
# Encontrar o primeiro dia do mês atual
colunas_agrupadas = ['COD_GRUPO_CLIENTE', 'DESC_GRUPO_E_CLIENTE', 'REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA']

hoje = datetime.today()
primeiro_dia_mes_atual = datetime(hoje.year, hoje.month, 1)

# Calcular o primeiro dia do mês de 6 meses atrás (excluindo mês atual)
primeiro_dia_6_meses_atras = (primeiro_dia_mes_atual - pd.DateOffset(months=6)).to_pydatetime()

# Filtrar apenas os últimos 6 meses (excluindo mês atual)
mask = (df_hist_vend_CLIENTE['PERIODO'] >= primeiro_dia_6_meses_atras) & (df_hist_vend_CLIENTE['PERIODO'] < primeiro_dia_mes_atual)
df_hist_vend_CLIENTE_ultimos_6_meses = df_hist_vend_CLIENTE.loc[mask].copy()

# Ordenar por data crescente
df_hist_vend_CLIENTE_ultimos_6_meses = df_hist_vend_CLIENTE_ultimos_6_meses.sort_values('PERIODO').reset_index(drop=True)

# DataFrame dos 3 meses mais recentes (últimos 3 meses do intervalo filtrado)
df_3_meses_mais_recentes = df_hist_vend_CLIENTE_ultimos_6_meses.copy()

# Identificar as 3 datas mais recentes (sem duplicar por linha)
meses_recentes = sorted(df_3_meses_mais_recentes['PERIODO'].unique())[-3:]

# Filtrar todas as linhas que pertencem a esses 3 meses
df_3_meses_mais_recentes = df_3_meses_mais_recentes[df_3_meses_mais_recentes['PERIODO'].isin(meses_recentes)].copy()

# Agrupa pelas colunas desejadas e calcula a média das colunas numéricas
df_3_meses_mais_recentes_media = df_3_meses_mais_recentes.groupby(colunas_agrupadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 3 MESES' na coluna
df_3_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 3 MESES'

# Agrupamento fazendo média dos 6 meses
df_6_meses_mais_recentes_media = df_hist_vend_CLIENTE_ultimos_6_meses.copy()
df_6_meses_mais_recentes_media = df_6_meses_mais_recentes_media.groupby(colunas_agrupadas).mean(numeric_only=True).reset_index()

# Adicionar coluna MEDIA informando 'MÉDIA 6 MESES' na coluna
df_6_meses_mais_recentes_media['MEDIA'] = 'MÉDIA 6 MESES'

# Concatenar os DataFrames
df_media_vendas_CLIENTE = pd.concat([df_3_meses_mais_recentes_media, df_6_meses_mais_recentes_media], ignore_index=True)

# Pivotar a coluna MEDIA
df_media_vendas_CLIENTE = df_media_vendas_CLIENTE.pivot_table(
    index=colunas_agrupadas,
    columns='MEDIA',
    values='VOL_VENDA',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Validar se as colunas 'MÉDIA 3 MESES' e 'MÉDIA 6 MESES' existem, caso contrário, criar com valor 0
if 'MÉDIA 3 MESES' not in df_media_vendas_CLIENTE.columns:
    df_media_vendas_CLIENTE['MÉDIA 3 MESES'] = 0.0
if 'MÉDIA 6 MESES' not in df_media_vendas_CLIENTE.columns:
    df_media_vendas_CLIENTE['MÉDIA 6 MESES'] = 0.0

# Gerar o arquivo CSV
df_media_vendas_CLIENTE.to_csv(
    pasta_input_painel / 'MEDIA_VENDA_KRONA_CLIENTE.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

del df_hist_vend_PRODUTO, df_hist_vend_CLIENTE, df_vendas_krona, produtos_a_eliminar
gc.collect()

print("✅ Bases de Vendas para Planejamento Colaborativo geradas com sucesso!")

✅ Bases de Vendas para Planejamento Colaborativo geradas com sucesso!


In [11]:
"""
🧩 Histórico dos Modelos Testados no Projeto
--------------------------------------------

1. Média Simples / Média 12M
   Descrição: cálculo da média das vendas dos últimos 12 meses.
   Objetivo: criar um ponto de partida rápido para testar estabilidade.
   Vantagem: extremamente leve e previsível.
   Limitação: ignora tendências (crescimento ou queda) e não reage a sazonalidades.

2. Média Móvel Ponderada
   Descrição: média dos últimos 12 meses com pesos maiores para os meses mais recentes.
   Objetivo: suavizar o histórico sem perder sensibilidade à tendência recente.
   Vantagem: melhora ligeiramente a resposta a movimentos recentes.
   Limitação: ainda não reconhece padrões anuais completos de sazonalidade.

3. Sazonalidade Percentual Histórica
   Descrição: calculava a participação média de cada mês no total anual.
   Objetivo: reproduzir o comportamento sazonal real da empresa.
   Vantagem: respeita picos e vales mensais do último ano completo.
   Limitação: dependente da qualidade do último ano — não projeta volume total, apenas distribui.

4. LightGBM
   Descrição: modelo de machine learning (boosting de árvores) aplicado sobre variáveis sazonais (seno/cosseno dos meses).
   Objetivo: prever volumes mensais aprendendo padrões não lineares.
   Vantagem: aprendizado rápido e robusto em bases amplas.
   Limitação: exige ajuste fino e mais dados; em séries curtas, tende a superajustar.

5. Regressão Linear (nível anual)
   Descrição: ajusta uma reta sobre as vendas anuais (y = a·x + b).
   Objetivo: capturar tendências de crescimento ou queda sustentadas.
   Vantagem: intuitivo e fácil de justificar visualmente.
   Limitação: não lida bem com oscilações bruscas ou séries curtas.

6. Holt-Winters Aditivo
   Descrição: modelo clássico de séries temporais com nível, tendência e sazonalidade (additive trend + seasonal).
   Objetivo: gerar previsões suaves mantendo padrão anual.
   Vantagem: reconhecido e equilibrado entre suavidade e tendência.
   Limitação: pesado em grandes volumes e instável em séries curtas.

7. Ensemble Estatístico (fase intermediária)
   Descrição: combinação ponderada de modelos simples (Regressão + Média + Suavização).
   Objetivo: estabilizar volumes sem perder aparência estatística.
   Vantagem: resultados consistentes e realistas.
   Limitação: apresentava sempre o mesmo nome, sem variação por empresa.
      
"""

"""
📊 Modelos de Previsão Aplicados (Versão Final)

Este script realiza previsão mensal por material (COD_PROD) utilizando múltiplos modelos estatísticos e de
Machine Learning. Para cada COD_PROD, os modelos são comparados via backtest no final do histórico e o modelo
com menor erro é selecionado automaticamente para gerar a projeção final.

Seleção do melhor modelo (por COD_PROD):
- O histórico é dividido em treino e validação (janela final, ex.: 12 meses).
- Cada modelo gera previsões para a janela de validação.
- O erro é calculado (WAPE ou MAPE, conforme configuração do script).
- O modelo com menor erro é escolhido para prever o horizonte futuro definido por:
  df_periodo_previsao["PERIODO_PROJECAO"] (lista de meses já tratados no upstream).

Modelos utilizados:

1) Regressão Linear (Linear Regression)
   - Captura: tendência linear ao longo do tempo (crescimento/queda constantes).
   - Uso: séries com comportamento estável ou como fallback quando há pouco histórico.
   - Força: simples, rápida, fácil de explicar e ótima como baseline.

2) Holt-Winters / Exponential Smoothing (com fallback)
   - Captura: tendência e, quando aplicável, sazonalidade (ex.: anual em dados mensais).
   - Uso: quando a série apresenta estrutura temporal consistente (tendência e/ou sazonalidade).
   - Força: modelo clássico e confiável para séries temporais; prioriza observações recentes.
   - Observação: o script tenta sazonalidade multiplicativa e/ou aditiva; se falhar, cai para Holt (trend-only).

3) ARIMA (AutoRegressive Integrated Moving Average)
   - Captura: autocorrelação, dinâmica temporal e ruído estatístico da série.
   - Uso: quando há padrão autoregressivo e boa estabilidade temporal.
   - Força: modelo estatístico tradicional, robusto e amplamente aceito em séries temporais.

4) Random Forest Regressor (Machine Learning)
   - Captura: padrões não lineares e interações entre variáveis temporais (ex.: mês/ano e tempo).
   - Uso: séries com comportamento irregular, não linear ou com variações que modelos lineares não capturam bem.
   - Força: robusto a ruídos, bom desempenho em cenários complexos e agrega “ML” ao portfólio técnico.

5) Gradient Boosting Regressor (Machine Learning)
   - Captura: relações complexas e refinamentos de padrão, frequentemente com boa precisão preditiva.
   - Uso: quando há não-linearidade, mudanças de regime e necessidade de um modelo ML mais “forte”.
   - Força: alto poder preditivo, muito usado em problemas de previsão com features temporais.
   
6) LinearRegression_Fallback (Estratégia de contingência)
   - Captura: tendência simples quando não há histórico suficiente para modelos mais robustos.
   - Uso: ativado automaticamente em COD_PROD com histórico curto ou quando os demais modelos falham.
   - Força: garante continuidade do processo sem interromper a geração da projeção.

Saída:
- A previsão final é incorporada ao dataset junto ao histórico.
- A coluna "MODELO_ESCOLHIDO" registra qual modelo foi selecionado para cada COD_PROD no horizonte previsto.
- A coluna de valor (ex.: "VOL_VENDA_REAL") contém histórico + projeção no mesmo padrão mensal.

OBSERVAÇÃO:
- O mesmo modelo que foi escolhido para prever o futuro foi testado nos últimos meses do histórico.
Isso nos permite medir a assertividade do modelo antes de usá-lo para projeção.
- Aplicado APE para identificar o erro de cada previsão definida no backtest. APE=(Real − Previsto)/Real
- Além da projeção, avaliamos a precisão histórica de cada modelo por produto, garantindo confiabilidade estatística nas previsões.

"""

'\n📊 Modelos de Previsão Aplicados (Versão Final)\n\nEste script realiza previsão mensal por material (COD_PROD) utilizando múltiplos modelos estatísticos e de\nMachine Learning. Para cada COD_PROD, os modelos são comparados via backtest no final do histórico e o modelo\ncom menor erro é selecionado automaticamente para gerar a projeção final.\n\nSeleção do melhor modelo (por COD_PROD):\n- O histórico é dividido em treino e validação (janela final, ex.: 12 meses).\n- Cada modelo gera previsões para a janela de validação.\n- O erro é calculado (WAPE ou MAPE, conforme configuração do script).\n- O modelo com menor erro é escolhido para prever o horizonte futuro definido por:\n  df_periodo_previsao["PERIODO_PROJECAO"] (lista de meses já tratados no upstream).\n\nModelos utilizados:\n\n1) Regressão Linear (Linear Regression)\n   - Captura: tendência linear ao longo do tempo (crescimento/queda constantes).\n   - Uso: séries com comportamento estável ou como fallback quando há pouco históric

In [12]:
# PREVISAO ESTATISTICA — 5 MODELOS + MELHOR POR COD_PROD (SEM TRATAMENTO EXTRA)
print("🔄 Iniciando processo de previsão estatística...")

# ============================================================
# 0) CARREGAR df_vendas_krona DO PARQUET (para limpar memória)
# ============================================================

df_vendas_krona = pd.read_parquet(pasta_staging_parquet / "df_vendas_krona.parquet")
print(f"📦 df_vendas_krona carregado | Linhas: {len(df_vendas_krona):,}")

# ============================================================
# 1) AGRUPAMENTO PADRÃO
# ============================================================

df_group = (
    df_vendas_krona
    .groupby(["COD_PROD", "PERIODO"], as_index=False)
    .agg(VOL_VENDA=("VOL_VENDA", "sum"))
    .sort_values(["COD_PROD", "PERIODO"])
)

print(
    f"📊 Dados agregados | SKUs: {df_group['COD_PROD'].nunique():,} | "
    f"Períodos: {df_group['PERIODO'].nunique():,}"
)

# ============================================================
# 2) CALENDÁRIO FUTURO
# ============================================================

future_dates = pd.DatetimeIndex(
    df_periodo_previsao["PERIODO_PROJECAO"].drop_duplicates().sort_values()
)

if len(future_dates) == 0:
    raise ValueError("df_periodo_previsao['PERIODO_PROJECAO'] está vazio.")

primeiro_mes_previsao = future_dates.min()
ultimo_mes_hist = primeiro_mes_previsao - pd.offsets.MonthBegin(1)

df_hist_base = df_group[df_group["PERIODO"] <= ultimo_mes_hist].copy()
if df_hist_base.empty:
    raise ValueError("Histórico vazio após corte pelo calendário futuro.")

horizon = len(future_dates)

print(
    f"🗓️ Horizonte de previsão | Meses: {horizon} | "
    f"{future_dates.min().date()} → {future_dates.max().date()}"
)

# ============================================================
# 3) MÉTRICA (WAPE ou MAPE)
# ============================================================

METRICA_USADA = "WAPE"  # "WAPE" recomendado; se quiser "MAPE", troque aqui.

def wape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return float(np.mean(np.abs(y_true - y_pred)))
    return float(np.sum(np.abs(y_true - y_pred)) / denom)

def safe_mape(y_true, y_pred) -> float:
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = y_true != 0
    if mask.sum() == 0:
        return float(np.mean(np.abs(y_true - y_pred)))
    return float(mean_absolute_percentage_error(y_true[mask], y_pred[mask]))

def metric(y_true, y_pred) -> float:
    return wape(y_true, y_pred) if METRICA_USADA == "WAPE" else safe_mape(y_true, y_pred)


# ============================================================
# 4) MODELOS
# ============================================================

def pred_hw(y_train, steps):
    # tenta sazonal; se falhar, cai pra Holt trend
    try:
        m = ExponentialSmoothing(y_train, trend="add", seasonal="multiplicative", seasonal_periods=12).fit()
        return np.maximum(m.forecast(steps), 0)
    except Exception:
        try:
            m = ExponentialSmoothing(y_train, trend="add", seasonal="additive", seasonal_periods=12).fit()
            return np.maximum(m.forecast(steps), 0)
        except Exception:
            m = ExponentialSmoothing(y_train, trend="add", seasonal=None).fit()
            return np.maximum(m.forecast(steps), 0)

def pred_arima(y_train, steps):
    m = ARIMA(y_train, order=(1,1,1)).fit()
    return np.maximum(m.forecast(steps), 0)

def pred_lr(y_train, steps):
    y_train = np.asarray(y_train, dtype=float)
    t = np.arange(len(y_train)).reshape(-1, 1)
    lr = LinearRegression().fit(t, y_train)
    t_future = np.arange(len(y_train), len(y_train) + steps).reshape(-1, 1)
    return np.maximum(lr.predict(t_future), 0)

def make_features_simple(y):
    # features MINIMAS pra ML sem depender de painel:
    # usa índice de tempo + mês (do próprio período)
    return y

def pred_rf(period_index_train, y_train, period_index_pred):
    # features: time + mês + ano
    X_train = pd.DataFrame({
        "time": np.arange(len(period_index_train)),
        "mes": period_index_train.month,
        "ano": period_index_train.year
    })
    rf = RandomForestRegressor(n_estimators=400, random_state=42)
    rf.fit(X_train, y_train)

    X_pred = pd.DataFrame({
        "time": np.arange(len(period_index_train), len(period_index_train) + len(period_index_pred)),
        "mes": period_index_pred.month,
        "ano": period_index_pred.year
    })
    return np.maximum(rf.predict(X_pred), 0)

def pred_gb(period_index_train, y_train, period_index_pred):
    X_train = pd.DataFrame({
        "time": np.arange(len(period_index_train)),
        "mes": period_index_train.month,
        "ano": period_index_train.year
    })
    gb = GradientBoostingRegressor(random_state=42)
    gb.fit(X_train, y_train)

    X_pred = pd.DataFrame({
        "time": np.arange(len(period_index_train), len(period_index_train) + len(period_index_pred)),
        "mes": period_index_pred.month,
        "ano": period_index_pred.year
    })
    return np.maximum(gb.predict(X_pred), 0)


# ============================================================
# 5) ESCOLHER MELHOR MODELO POR COD_PROD (backtest) + prever futuro
# ============================================================

JANELA_VALIDACAO = 12

registros = []
best_model_por_sku = {}  # guardar o modelo escolhido por COD_PROD (para backtest completo)

total_skus = df_hist_base["COD_PROD"].nunique()
print(f"🚀 Iniciando previsão por SKU | Total: {total_skus:,}")

for i, (cod_prod, df_sku) in enumerate(df_hist_base.groupby("COD_PROD"), start=1):

    if i == 1 or i % 50 == 0:
        print(f"   ▶️ Processando SKU {i}/{total_skus} | COD_PROD={cod_prod}")

    df_sku = df_sku.sort_values("PERIODO")
    y = df_sku["VOL_VENDA"].values.astype(float)
    idx = pd.DatetimeIndex(df_sku["PERIODO"])

    # histórico muito curto -> LR
    if len(y) < 6:
        fc = pred_lr(y, horizon)
        best = "LinearRegression_Fallback"
        best_model_por_sku[cod_prod] = best
        for per, val in zip(future_dates, fc):
            registros.append([cod_prod, per, float(val), best])
        continue

    J = min(JANELA_VALIDACAO, max(3, len(y)//3))
    y_train, y_val = y[:-J], y[-J:]
    idx_train, idx_val = idx[:-J], idx[-J:]

    scores = {}

    # 1) HW
    try:
        pred_val = pred_hw(y_train, J)
        scores["HoltWinters"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 2) ARIMA
    try:
        pred_val = pred_arima(y_train, J)
        scores["ARIMA"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 3) LR
    try:
        pred_val = pred_lr(y_train, J)
        scores["LinearRegression"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 4) RF (ML)
    try:
        pred_val = pred_rf(idx_train, y_train, idx_val)
        scores["RandomForest"] = metric(y_val, pred_val)
    except Exception:
        pass

    # 5) GB (ML)
    try:
        pred_val = pred_gb(idx_train, y_train, idx_val)
        scores["GradientBoosting"] = metric(y_val, pred_val)
    except Exception:
        pass

    # escolher melhor e treinar no histórico completo
    if not scores:
        best = "LinearRegression_Fallback"
        fc = pred_lr(y, horizon)
    else:
        best = min(scores.items(), key=lambda x: x[1])[0]

        if best == "HoltWinters":
            fc = pred_hw(y, horizon)
        elif best == "ARIMA":
            fc = pred_arima(y, horizon)
        elif best == "LinearRegression":
            fc = pred_lr(y, horizon)
        elif best == "RandomForest":
            fc = pred_rf(idx, y, future_dates)
        else:  # GradientBoosting
            fc = pred_gb(idx, y, future_dates)

    best_model_por_sku[cod_prod] = best

    for per, val in zip(future_dates, fc):
        registros.append([cod_prod, per, float(val), best])

print("🧩 Finalizando consolidação de resultados (forecast futuro)...")

df_forecast = pd.DataFrame(registros, columns=["COD_PROD", "PERIODO", "VOL_VENDA_REAL", "MODELO_ESCOLHIDO"])


# ============================================================
# 6) SAÍDA FINAL (histórico + projeção)
# ============================================================

df_final_hist = df_hist_base.rename(columns={"VOL_VENDA": "VOL_VENDA_REAL"}).copy()
df_final_hist["MODELO_ESCOLHIDO"] = np.nan

df_forecast_estatistico_krona = pd.concat([df_final_hist, df_forecast], ignore_index=True)

print("📦 Dataset base montado (histórico + futuro).")

# ============================================================
# 6.1) BACKTEST COMPLETO (walk-forward) com o MESMO modelo do futuro
#      Preenche o histórico inteiro (a partir de MIN_TREINO)
# ============================================================

print("🧪 Iniciando backtest completo (walk-forward) por SKU...")

STEP_BACKTEST = 6   # Treina a cada 6 meses

# criar colunas novas (SEM criar APE_PCT / MAPE_SKU_PCT)
df_forecast_estatistico_krona["PREVISAO_BACKTEST"] = np.nan
df_forecast_estatistico_krona["MODELO_BACKTEST"] = np.nan
df_forecast_estatistico_krona["APE"] = np.nan
df_forecast_estatistico_krona["MAPE_SKU"] = np.nan

MIN_TREINO = 12  # mínimo de meses para começar a prever o passado (walk-forward)

# vamos trabalhar no histórico apenas (onde MODELO_ESCOLHIDO é NaN)
df_hist_all = df_forecast_estatistico_krona[df_forecast_estatistico_krona["MODELO_ESCOLHIDO"].isna()].copy()
df_hist_all = df_hist_all.sort_values(["COD_PROD", "PERIODO"])

total_skus_bt = df_hist_all["COD_PROD"].nunique()
print(f"🔎 Backtest completo | Total SKUs: {total_skus_bt:,} | MIN_TREINO={MIN_TREINO}")

mape_por_sku_final = {}  # MAPE final calculado pelo backtest completo

for i, (cod_prod, d) in enumerate(df_hist_all.groupby("COD_PROD"), start=1):

    if i == 1 or i % 50 == 0:
        print(f"   ▶️ Backtest SKU {i}/{total_skus_bt} | COD_PROD={cod_prod}")

    d = d.sort_values("PERIODO")
    y = d["VOL_VENDA_REAL"].values.astype(float)
    idx = pd.DatetimeIndex(d["PERIODO"])

    best = best_model_por_sku.get(cod_prod, "LinearRegression_Fallback")

    preds = np.full(len(y), np.nan, dtype=float)
    ape = np.full(len(y), np.nan, dtype=float)

    t = MIN_TREINO

    while t < len(y):
        y_train = y[:t]
        idx_train = idx[:t]

        steps = min(STEP_BACKTEST, len(y) - t)
        idx_pred = idx[t:t + steps]

        try:
            if best == "HoltWinters":
                y_preds = pred_hw(y_train, steps)
            elif best == "ARIMA":
                y_preds = pred_arima(y_train, steps)
            elif best in ("LinearRegression", "LinearRegression_Fallback"):
                y_preds = pred_lr(y_train, steps)
            elif best == "RandomForest":
                y_preds = pred_rf(idx_train, y_train, idx_pred)
            else:  # GradientBoosting
                y_preds = pred_gb(idx_train, y_train, idx_pred)
        except Exception:
            y_preds = np.full(steps, np.mean(y_train) if len(y_train) else 0.0)

        for i_step in range(steps):
            pos = t + i_step
            preds[pos] = max(float(y_preds[i_step]), 0)

            if y[pos] != 0:
                ape[pos] = abs((y[pos] - preds[pos]) / y[pos])

        t += steps


    mape_sku = float(np.nanmean(ape))
    mape_por_sku_final[cod_prod] = mape_sku

    # escrever de volta no df final (somente linhas históricas do SKU)
    mask = (df_forecast_estatistico_krona["COD_PROD"] == cod_prod) & (df_forecast_estatistico_krona["MODELO_ESCOLHIDO"].isna())

    # garantir alinhamento: como d está em ordem e mask pega as linhas do SKU no mesmo bloco histórico,
    # vamos atribuir pela ordem (via values) para evitar problemas com índice
    idx_rows = df_forecast_estatistico_krona.loc[mask].sort_values("PERIODO").index

    df_forecast_estatistico_krona.loc[idx_rows, "PREVISAO_BACKTEST"] = preds
    df_forecast_estatistico_krona.loc[idx_rows, "MODELO_BACKTEST"] = best
    df_forecast_estatistico_krona.loc[idx_rows, "APE"] = ape
    df_forecast_estatistico_krona.loc[idx_rows, "MAPE_SKU"] = mape_sku

# preencher MAPE_SKU também no futuro (indicador por SKU)
df_forecast_estatistico_krona["MAPE_SKU"] = df_forecast_estatistico_krona["MAPE_SKU"].fillna(
    df_forecast_estatistico_krona["COD_PROD"].map(mape_por_sku_final)
)

# Criar coluna PREVISAO_FINAL baseada em PREVISAO_BACKTEST, porém nos periodos futuros usar VOL_VENDA_REAL
df_forecast_estatistico_krona["PREVISAO_FINAL"] = np.where(
    df_forecast_estatistico_krona["PERIODO"].isin(future_dates),
    df_forecast_estatistico_krona["VOL_VENDA_REAL"],
    df_forecast_estatistico_krona["PREVISAO_BACKTEST"]
)

print("✅ Backtest completo concluído (histórico preenchido).")

# ============================================================
# 7) SALVAR CSV
# ============================================================

df_forecast_estatistico_krona.to_csv(
    pasta_staging_parquet / "df_forecast_estatistico_krona.csv",
    sep=";",
    encoding="utf-8-sig",
    index=False,
    decimal=",",
    float_format="%.2f"
)

print("✅ Finalizado e salvo: df_forecast_estatistico_krona.csv")

🔄 Iniciando processo de previsão estatística...
📦 df_vendas_krona carregado | Linhas: 11,105,253
📊 Dados agregados | SKUs: 1,180 | Períodos: 49
🗓️ Horizonte de previsão | Meses: 4 | 2026-03-01 → 2026-06-01
🚀 Iniciando previsão por SKU | Total: 1,180
   ▶️ Processando SKU 1/1180 | COD_PROD=0023
   ▶️ Processando SKU 50/1180 | COD_PROD=0152
   ▶️ Processando SKU 100/1180 | COD_PROD=0240
   ▶️ Processando SKU 150/1180 | COD_PROD=0313
   ▶️ Processando SKU 200/1180 | COD_PROD=0378
   ▶️ Processando SKU 250/1180 | COD_PROD=0432
   ▶️ Processando SKU 300/1180 | COD_PROD=0484
   ▶️ Processando SKU 350/1180 | COD_PROD=0561
   ▶️ Processando SKU 400/1180 | COD_PROD=0621
   ▶️ Processando SKU 450/1180 | COD_PROD=0677
   ▶️ Processando SKU 500/1180 | COD_PROD=0738
   ▶️ Processando SKU 550/1180 | COD_PROD=0806
   ▶️ Processando SKU 600/1180 | COD_PROD=0928
   ▶️ Processando SKU 650/1180 | COD_PROD=1031
   ▶️ Processando SKU 700/1180 | COD_PROD=1143
   ▶️ Processando SKU 750/1180 | COD_PROD=1232
 

In [13]:
parar_execucao()

AssertionError: Execução interrompida DEBUG.

In [67]:
# ============================================================
# DESAGREGAÇÃO DO FORECAST ESTATÍSTICO (HISTÓRICO + FUTURO)
# SAÍDA ÚNICA: df_prev_krona.parquet (HIST + FUT)
# ============================================================

# =========================
# START
# =========================
print("🔄 Iniciando processo de desagregação (histórico + futuro)...")
print(f"📦 df_vendas_krona em memória | Linhas: {len(df_vendas_krona):,}".replace(",", "."))

# =========================
# LER FORECAST
# =========================
print("📊 Lendo arquivo de previsão estatística salvo...")

arquivo_forecast = pasta_staging_parquet / "df_forecast_estatistico_krona.csv"

df_forecast_estatistico_krona = pd.read_csv(
    arquivo_forecast,
    sep=";",
    encoding="utf-8-sig",
    decimal=",",
    dtype={"COD_PROD": str},
    dayfirst=True
)

df_forecast_estatistico_krona["PERIODO"] = pd.to_datetime(df_forecast_estatistico_krona["PERIODO"])
df_vendas_krona["PERIODO"] = pd.to_datetime(df_vendas_krona["PERIODO"])

# Eliminar colunas desnecessárias no df_forecast_estatistico_krona
df_forecast_estatistico_krona = df_forecast_estatistico_krona.drop(
    columns=["MODELO_ESCOLHIDO", "PREVISAO_BACKTEST", "MODELO_BACKTEST", "APE", "MAPE_SKU", "VOL_VENDA_REAL"]
)

# Manter somente PERIODO conforme variável df_periodo_previsao
df_forecast_estatistico_krona = df_forecast_estatistico_krona[
    df_forecast_estatistico_krona["PERIODO"].isin(
        pd.to_datetime(df_periodo_previsao["PERIODO_PROJECAO"].unique())
    )
].copy().reset_index(drop=True)

n_skus = df_forecast_estatistico_krona["COD_PROD"].nunique()
n_periodos = df_forecast_estatistico_krona["PERIODO"].nunique()
p_min = df_forecast_estatistico_krona["PERIODO"].min()
p_max = df_forecast_estatistico_krona["PERIODO"].max()

print(f"📊 Forecast carregado | SKUs: {n_skus:,} | Períodos: {n_periodos}".replace(",", "."))
print(f"🗓️ Horizonte de previsão | Meses: {n_periodos} | {p_min.date()} → {p_max.date()}")

# =========================
# DEFINIR HISTÓRICO PARA DESAGREGAÇÃO DO FORECAST
# =========================
print("📦 Montando base df_prev_krona (histórico)...")
meses_hist_desagregacao = 12

# df_prev_krona deve ser cópia de df_vendas_krona, filtrando PERIODO pela variavel meses_hist_desagregacao, retornar os ultimos 12 que constam no arquivo df_vendas_krona
periodos_disponiveis = sorted(df_vendas_krona["PERIODO"].unique())
periodos_para_manter = periodos_disponiveis[-meses_hist_desagregacao:]
df_prev_krona = df_vendas_krona[df_vendas_krona["PERIODO"].isin(periodos_para_manter)].copy().reset_index(drop=True)

# Agrupar dados somando as VOL_VENDA
chaves_desagregacao = [
    "EMPRESA","COD_CLIENTE","NOME_CLIENTE","COD_GRUPO_CLIENTE","DESC_GRUPO_E_CLIENTE",
    "COD_PROD","DESC_PRODUTO","FAMILIA","LINHA","REGIONAL","REGIONAL_GESTOR"
]
chaves_sem_periodo = chaves_desagregacao[:]
df_prev_krona = df_prev_krona.groupby(
    chaves_desagregacao,
    as_index=False
).agg({'VOL_VENDA': 'sum'}).reset_index(drop=True)

print("🚀 Gerando percentuais de desagregação")

# Criar coluna TOTAL_VOL_VENDA por COD_PROD
total_vol_venda_por_prod = df_prev_krona.groupby('COD_PROD')['VOL_VENDA'].transform('sum')
df_prev_krona["TOTAL_VOL_VENDA"] = total_vol_venda_por_prod

# Criar coluna PERC_DESAGR
df_prev_krona["PERC_DESAGR"] = df_prev_krona["VOL_VENDA"] / df_prev_krona["TOTAL_VOL_VENDA"]

df_prev_explodido = (
    df_prev_krona
    .merge(
        df_forecast_estatistico_krona,
        on="COD_PROD",
        how="inner"   # só explode onde existe forecast
    )
)

print("🚀 Aplicando percentuais para desagregar o forecast estatístico...")

df_prev_explodido["VOL_PREV"] = (
    df_prev_explodido["PREVISAO_FINAL"] * df_prev_explodido["PERC_DESAGR"]
)

df_prev_krona = df_prev_explodido.copy()

# FIXME
del df_prev_explodido, df_forecast_estatistico_krona, df_vendas_krona
gc.collect()

print(f"✅ df_prev_krona pronto | Linhas: {len(df_prev_krona):,}".replace(",", "."))

print("💾 Salvando df_prev_krona completo em Parquet...")

df_prev_krona.to_parquet(
    pasta_staging_parquet / "df_prev_krona.parquet",
    engine="pyarrow",
    compression="snappy",
    index=False
)

print("✅ Parquet salvo com sucesso: df_prev_krona.parquet")

del df_prev_krona
gc.collect()


🔄 Iniciando processo de desagregação (histórico + futuro)...
📦 df_vendas_krona em memória | Linhas: 11.105.253
📊 Lendo arquivo de previsão estatística salvo...
📊 Forecast carregado | SKUs: 1.180 | Períodos: 4
🗓️ Horizonte de previsão | Meses: 4 | 2026-03-01 → 2026-06-01
📦 Montando base df_prev_krona (histórico)...
🚀 Gerando percentuais de desagregação
🚀 Aplicando percentuais para desagregar o forecast estatístico...
✅ df_prev_krona pronto | Linhas: 5.546.032
💾 Salvando df_prev_krona completo em Parquet...
✅ Parquet salvo com sucesso: df_prev_krona.parquet


0

In [ ]:
# Separar a df_forecast_vendas_krona em dois dataframes:
# df_forecast_vendas_krona_CLIENTE: clientes que terão planejamento de demanda
# df_forecast_vendas_krona_PRODUTO: produtos que terão planejamento de demanda

print("🔄 Gerando arquivos Forecast para Painel S&OP...")

# Ler df_prev_krona do Parquet
df_prev_krona = pd.read_parquet(pasta_staging_parquet / "df_prev_krona.parquet")

df_forecast_vendas_krona = df_prev_krona[
    df_prev_krona["PERIODO"].isin(df_periodo_previsao["PERIODO_PROJECAO"])
].copy()

# Se lista_clientes_plan_demanda estiver vazio → todos são PRODUTO
if lista_clientes_plan_demanda and len(lista_clientes_plan_demanda) > 0:
    df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] = np.where(
        df_forecast_vendas_krona['COD_GRUPO_CLIENTE'].isin(lista_clientes_plan_demanda),
        'CLIENTE',
        'PRODUTO'
    )
else:
    # Se não existe cliente para plan. demanda → tudo produto
    df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] = 'PRODUTO'
    
# Separar os dataframes com cópia explícita
df_forecast_vendas_krona_CLIENTE = df_forecast_vendas_krona[df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'CLIENTE'].copy()
df_forecast_vendas_krona_PRODUTO = df_forecast_vendas_krona[df_forecast_vendas_krona['NIVEL_PLAN_DEMANDA'] == 'PRODUTO'].copy()

# Eliminar coluna NIVEL_PLAN_DEMANDA
df_forecast_vendas_krona_CLIENTE.drop(columns=['NIVEL_PLAN_DEMANDA'], inplace=True)
df_forecast_vendas_krona_CLIENTE.reset_index(drop=True, inplace=True)
df_forecast_vendas_krona_PRODUTO.drop(columns=['NIVEL_PLAN_DEMANDA'], inplace=True)
df_forecast_vendas_krona_PRODUTO.reset_index(drop=True, inplace=True)

#========================================================
# PADRONIZANDO TEMPLATE df_forecast_vendas_krona_PRODUTO
#========================================================

# Eliminar colunas NOME_CLIENTE E DESC_GRUPO_E_CLIENTE
df_forecast_vendas_krona_PRODUTO.drop(columns=['COD_CLIENTE', 'NOME_CLIENTE', 'COD_GRUPO_CLIENTE', 'DESC_GRUPO_E_CLIENTE'], inplace=True)

# Sumarizar df_forecast_vendas_krona_PRODUTO por EMPRESA, COD_PROD, DESC_PRODUTO, FAMILIA, LINHA, REGIONAL, PERIODO
df_forecast_vendas_krona_PRODUTO = df_forecast_vendas_krona_PRODUTO.groupby(
    ['EMPRESA', 'COD_PROD', 'DESC_PRODUTO', 'FAMILIA', 'LINHA', 'REGIONAL', 'REGIONAL_GESTOR', 'PERIODO'],
    as_index=False
).agg({'VOL_PREV': 'sum'}).reset_index(drop=True)

# Gerar arquivos em PARQUET
df_forecast_vendas_krona_PRODUTO.to_parquet(pasta_staging_parquet / 'df_forecast_vendas_krona_PRODUTO.parquet', index=False)
df_forecast_vendas_krona_CLIENTE.to_parquet(pasta_staging_parquet / 'df_forecast_vendas_krona_CLIENTE.parquet', index=False)

# 📤 Exportação de Dados Forecast para Planejamento Colaborativo
# 📊 Nível de agregação: REGIONAL_GESTOR, 'REGIONAL', 'FAMILIA', 'PERIODO'

df_Forecast_PRODUTO = df_forecast_vendas_krona_PRODUTO.groupby(
    ['REGIONAL_GESTOR', 'REGIONAL', 'FAMILIA', 'PERIODO'],
    as_index=False
).agg({'VOL_PREV': 'sum'}).reset_index(drop=True)

df_Forecast_PRODUTO.to_csv(
    pasta_input_painel / 'FORECAST_KRONA_AGREGADO.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

print("✅ Arquivo FORECAST_KRONA_AGREGADO.csv gerado com sucesso!")

#========================================================
# PADRONIZANDO TEMPLATE df_forecast_vendas_krona_CLIENTE
#========================================================

# 📤 Exportação de Dados Forecast para Planejamento Colaborativo
# 📊 Nível de agregação: REGIONAL_GESTOR, 'REGIONAL', 'COD_GRUPO_CLIENTE, DESC_GRUPO_E_CLIENTE, FAMILIA e PERIODO

df_Forecast_CLIENTE = df_forecast_vendas_krona_CLIENTE.groupby(
    ['REGIONAL_GESTOR', 'REGIONAL', 'COD_GRUPO_CLIENTE', 'DESC_GRUPO_E_CLIENTE', 'FAMILIA', 'PERIODO'],
    as_index=False
).agg({'VOL_PREV': 'sum'}).reset_index(drop=True)

df_Forecast_CLIENTE.to_csv(
    pasta_input_painel / 'FORECAST_KRONA_CLIENTE.csv',
    sep=';',
    encoding='utf-8-sig',
    index=False,
    decimal=',',
    float_format="%.2f"
)

print("✅ Arquivo FORECAST_KRONA_CLIENTE.csv gerado com sucesso!")

del df_forecast_vendas_krona_PRODUTO, df_forecast_vendas_krona_CLIENTE, lista_clientes_plan_demanda, df_prev_krona
gc.collect()

timer.finalizar()
print("🎯 Processo concluído com sucesso!")

🔄 Gerando arquivos Forecast para Painel S&OP...
✅ Arquivo FORECAST_KRONA_AGREGADO.csv gerado com sucesso!
✅ Arquivo FORECAST_KRONA_CLIENTE.csv gerado com sucesso!

⏱️ Tempo total de processamento: 368 min 23.8 s
🎯 Processo concluído com sucesso!
